这个notebook的作用是根据合成的vnpy格式的日线主力连续合约, 生成1分钟线的vnpy格式主力连续合约

In [159]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import time
import json
import logging
%matplotlib inline

In [160]:
symbol_info = pd.read_csv(r"E:\project\trade_engine\iqfeed_data_processor\project_data\my_symbols.csv")
symbol_info.head()

,description,iqfeed_roots,ib_root,symbol_exchange,commission_ratio,commission,tick_value,tick_price,size,rough_price,contract_value,init_margin,maintain_margin
0,S&P 500 E-Mini,@ES,ES,GLOBEX,0.000015,2.10,12.50,0.25,50,2841.0,142050,13200.00,12000.00
1,S&P 500 E-micro,@MES,MES,GLOBEX,0.000033,0.47,1.25,0.25,5,2841.0,14205,1320.00,1200.00
2,Nasdaq 100 E-micro,@MNQ,MNQ,GLOBEX,0.000026,0.47,0.50,0.25,2,9038.0,18076,1652.81,1502.55
3,Dow Futures micro,@MYM,MYM,CBT,0.000020,0.47,0.50,0.50,1,23597.0,23597,3608.00,3137.39
4,Russell 2000 E-micro,@M2K,M2K,GLOBEX,0.000075,0.47,0.50,0.10,5,1257.0,6285,980.45,784.36


In [161]:
with open(r"E:\project\trade_engine\iqfeed_data_processor\project_data\trade_time.json") as json_file:
    trade_time_dict = json.load(json_file)

# 需要找到一周每天的每个合约的开盘时间开始和结束. key是周几,value是开盘时间和收盘时间的一个list
# 0是周一.... 6是周日

In [162]:
day_continuous_folder = r"D:\data\vnpy\future\day_continuous"
one_min_continuous_folder = r"D:\data\vnpy\future\1_min_continuous"

In [163]:
for handler in logger.handlers[:]:
    handler.close()
    logger.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger("day_to_1min_continuous")
currentDT = datetime.now()
handler = logging.FileHandler("day_to_1min_continuous" + currentDT.strftime("%Y_%m_%d_%H_%M_%S") + ".log")
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

for ix, row in symbol_info.iterrows():
    ib_root = row.loc["ib_root"]
    # 先要找到trade time时间的info 
    try:
        market_open_time = trade_time_dict[ib_root]
    except Exception:
        logger.error("{} is not found in trade time info json file".format(ib_root))
        continue
    # 需要日线的主力连续合约的csv file
    day_continuous_path = os.path.join(day_continuous_folder, ib_root + ".csv")
    if not os.path.exists(day_continuous_path):
        logger.warning("{} day continuous data not found".format(ib_root))
        continue
    else:
        logger.info("Starting processing data for {}".format(ib_root))
    
    #如果1分钟线主力连续合约已经存在,则暂时不重新弄了
    one_min_continuous_path = os.path.join(one_min_continuous_folder, ib_root + ".csv")
    if os.path.exists(one_min_continuous_path):
        logger.info("{} one min continuous data already exists".format(ib_root))
        continue
        
    day_continuous_frame = pd.read_csv(day_continuous_path)
    day_continuous_frame.loc[:, "date"] = pd.to_datetime(day_continuous_frame['date'], format='%Y-%m-%d')
    day_continuous_frame = day_continuous_frame.sort_values("date")
    day_continuous_frame.loc[:, "weekday"] = day_continuous_frame["date"].apply(lambda x: x.weekday())
    one_min_data_paths = glob.glob(r"D:\data\vnpy\future\1_min\*.csv")
    one_min_data_paths = [path for path in one_min_data_paths if ib_root == os.path.basename(path)[: -7]]
    one_min_data_paths = sorted(one_min_data_paths)
    one_min_data_paths = sorted(one_min_data_paths, key=lambda x: x[-6:-4])
    #读取并且缓存所有的1min frame, key是symbol, value是data frame
    logger.info("Reading all data to cache.")
    one_min_data_dict = {}
    for path in one_min_data_paths:
        symbol = os.path.basename(path)[:-4]
        frame = pd.read_csv(path)
        frame.loc[:, "datetime"] = pd.to_datetime(frame['datetime'], format='%Y%m%d %H:%M:%S')
        frame = frame.sort_values("datetime")
        one_min_data_dict[symbol] = frame

    #下面的Code开始拼接主力合约的1分钟bar数据
    #循环day bar frame
    start_time = time.time()
    # 搞一个用来存储每一个subframe的list
    all_sub_frames = []
    for ix, row in day_continuous_frame.iterrows():
        if ix%500 == 0:
            logger.info("process: {}/{}".format(ix, len(day_continuous_frame)))

        date = row.loc["date"]
        weekday = row.loc["weekday"]
        # 有一些期货2015年9月19号之前变动过交易时间, 有单独的tradetime info的格式.
        if "current_time" in market_open_time.keys():
            weekday_list = market_open_time["current_time"][str(weekday)]
        else:
            if date < datetime(2015,9,19):
                weekday_list = market_open_time["pre_2015_09_18"][str(weekday)]
            else:
                weekday_list = market_open_time["after_2015_09_18"][str(weekday)]
        # 要找到交易时段的所有每一分钟
        for start_end_list in weekday_list:
            min_bar_start_time = datetime(date.year,
                                          date.month,
                                          date.day,
                                          start_end_list[0][0],
                                          start_end_list[0][1],
                                          start_end_list[0][2])
            min_bar_end_time = datetime(date.year,
                                        date.month,
                                        date.day,
                                        start_end_list[1][0],
                                        start_end_list[1][1],
                                        start_end_list[1][2])
            # find all one min timestamps between a start time and an end time
            one_min_time_range = pd.date_range(min_bar_start_time, min_bar_end_time, freq='min')
            # 新建一个one min frame
            new_one_min_frame = pd.DataFrame(columns=day_continuous_frame.columns, index=one_min_time_range)
            # 先把能填充的基础数据都填充了
            symbol = row.loc["symbol"]
            # 找到对应的symbol的实际数据frame
            symbol_frame = one_min_data_dict[symbol]
            # 把symbol data frame 的 index变为找到所有的minute的数据, 其他的数据暂时不要
            new_one_min_frame = symbol_frame.set_index("datetime").reindex(new_one_min_frame.index)
            #节假日某个半天可能没有交易, 不能要这个时间段的数据
            if new_one_min_frame.isnull().all().all():
                logger.info("weekday:{}".format(weekday))
                logger.info("exclude holiday: {}, symbol: {}".format(min_bar_end_time, symbol))
            else:
                new_one_min_frame.loc[:, "accumulative_adjust"] = row.loc["accumulative_adjust"]
                price_columns = ["open", "high", "low", "close"]
                new_one_min_frame.loc[:, price_columns] = new_one_min_frame.loc[:, price_columns] - row.loc["accumulative_adjust"]
                new_one_min_frame.loc[:, "exchange"] = row.loc["exchange"]
                new_one_min_frame.loc[:, "open_interest"] = row.loc["open_interest"]
                new_one_min_frame.loc[:, "interval"] = row.loc["interval"]
                new_one_min_frame.loc[:, "symbol"] = row.loc["symbol"]
                all_sub_frames.append(new_one_min_frame)


    #1min continuous frame 就是把所有的sub frame拼起来
    one_min_continuous_frame = pd.concat(all_sub_frames)

    #所有的空值全部由前一个bar的close的值向后填充,和ib数据保持一致.
    close_ffill = one_min_continuous_frame.loc[:, "close"].fillna(method='ffill')
    fill_nan_values = close_ffill

    for column in ["open", "high", "low", "close"]:
        one_min_continuous_frame.loc[:, column] = one_min_continuous_frame.loc[:, column].fillna(fill_nan_values)

    #填充的volume应该是0,因为没交易.
    one_min_continuous_frame.loc[:, "volume"] = one_min_continuous_frame.loc[:, "volume"].fillna(0)

    #最开始的空的bar不要了,没有办法填充
    one_min_continuous_frame = one_min_continuous_frame.dropna()

    #最后一天的数据不要,因为不完整
    last_date = one_min_continuous_frame.index[-1].date()
    last_date_start = datetime(last_date.year, last_date.month, last_date.day)
    one_min_continuous_frame.drop(one_min_continuous_frame.index[one_min_continuous_frame.index >=  last_date_start], inplace=True)

    
    one_min_continuous_frame.to_csv(one_min_continuous_path)
    end_time = time.time()
    logger.info("total time used is: {}".format(end_time - start_time))

2020-06-28 21:32:51,992 - day_to_1min_continuous - INFO - Starting processing data for ES
2020-06-28 21:32:52,055 - day_to_1min_continuous - INFO - Reading all data to cache.
2020-06-28 21:33:14,466 - day_to_1min_continuous - INFO - process: 0/3954
2020-06-28 21:33:14,517 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:33:14,519 - day_to_1min_continuous - INFO - exclude holiday: 2007-09-20 16:00:00, symbol: ESH08
2020-06-28 21:33:14,538 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:33:14,540 - day_to_1min_continuous - INFO - exclude holiday: 2007-09-20 23:59:00, symbol: ESH08
2020-06-28 21:33:14,673 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:33:14,675 - day_to_1min_continuous - INFO - exclude holiday: 2007-09-24 16:00:00, symbol: ESH08
2020-06-28 21:33:14,775 - day_to_1min_continuous - INFO - weekday:1
2020-06-28 21:33:14,777 - day_to_1min_continuous - INFO - exclude holiday: 2007-09-25 23:59:00, symbol: ESH08
2020-06-28 21:33:18,015 - day_to_1min_c

2020-06-28 21:33:42,420 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 21:33:42,422 - day_to_1min_continuous - INFO - exclude holiday: 2008-12-24 23:59:00, symbol: ESH09
2020-06-28 21:33:42,819 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 21:33:42,821 - day_to_1min_continuous - INFO - exclude holiday: 2008-12-31 23:59:00, symbol: ESH09
2020-06-28 21:33:43,962 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:33:43,963 - day_to_1min_continuous - INFO - exclude holiday: 2009-01-19 16:00:00, symbol: ESH09
2020-06-28 21:33:45,918 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:33:45,920 - day_to_1min_continuous - INFO - exclude holiday: 2009-02-16 16:00:00, symbol: ESH09
2020-06-28 21:33:49,639 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:33:49,640 - day_to_1min_continuous - INFO - exclude holiday: 2009-04-09 23:59:00, symbol: ESM09
2020-06-28 21:33:51,192 - day_to_1min_continuous - INFO - process: 500/3954
2020-06-28 21:33:52,678 - day_to_1

2020-06-28 21:35:35,854 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:35:35,856 - day_to_1min_continuous - INFO - exclude holiday: 2013-05-27 16:00:00, symbol: ESM13
2020-06-28 21:35:38,403 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 21:35:38,405 - day_to_1min_continuous - INFO - exclude holiday: 2013-07-03 16:00:00, symbol: ESU13
2020-06-28 21:35:38,493 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:35:38,494 - day_to_1min_continuous - INFO - exclude holiday: 2013-07-04 16:00:00, symbol: ESU13
2020-06-28 21:35:42,599 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:35:42,601 - day_to_1min_continuous - INFO - exclude holiday: 2013-09-02 16:00:00, symbol: ESU13
2020-06-28 21:35:48,569 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:35:48,571 - day_to_1min_continuous - INFO - exclude holiday: 2013-11-28 16:00:00, symbol: ESZ13
2020-06-28 21:35:48,675 - day_to_1min_continuous - INFO - weekday:4
2020-06-28 21:35:48,677 - day_to_1min_cont

2020-06-28 21:37:05,272 - day_to_1min_continuous - INFO - exclude holiday: 2017-01-16 16:00:00, symbol: ESH17
2020-06-28 21:37:07,557 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:37:07,559 - day_to_1min_continuous - INFO - exclude holiday: 2017-02-20 16:00:00, symbol: ESH17
2020-06-28 21:37:11,046 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:37:11,047 - day_to_1min_continuous - INFO - exclude holiday: 2017-04-13 23:59:00, symbol: ESM17
2020-06-28 21:37:13,316 - day_to_1min_continuous - INFO - process: 3000/3954
2020-06-28 21:37:13,858 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:37:13,860 - day_to_1min_continuous - INFO - exclude holiday: 2017-05-29 16:00:00, symbol: ESM17
2020-06-28 21:37:16,321 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:37:16,322 - day_to_1min_continuous - INFO - exclude holiday: 2017-07-03 16:00:00, symbol: ESU17
2020-06-28 21:37:16,411 - day_to_1min_continuous - INFO - weekday:1
2020-06-28 21:37:16,412 - day_to_

2020-06-28 21:38:17,558 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:38:17,560 - day_to_1min_continuous - INFO - exclude holiday: 2020-01-20 16:00:00, symbol: ESH20
2020-06-28 21:38:19,322 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:38:19,324 - day_to_1min_continuous - INFO - exclude holiday: 2020-02-17 16:00:00, symbol: ESH20
2020-06-28 21:38:22,820 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:38:22,822 - day_to_1min_continuous - INFO - exclude holiday: 2020-04-09 23:59:00, symbol: ESM20
2020-06-28 21:38:25,612 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:38:25,614 - day_to_1min_continuous - INFO - exclude holiday: 2020-05-25 16:00:00, symbol: ESM20
2020-06-28 21:38:26,875 - day_to_1min_continuous - INFO - weekday:4
2020-06-28 21:38:26,877 - day_to_1min_continuous - INFO - exclude holiday: 2020-06-12 16:00:00, symbol: ESM20
2020-06-28 21:39:36,424 - day_to_1min_continuous - INFO - total time used is: 382.27707147598267
2020-06-28 21

...

2020-06-28 21:54:28,858 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:54:28,860 - day_to_1min_continuous - INFO - exclude holiday: 2012-05-03 23:59:00, symbol: CTV12
2020-06-28 21:54:28,902 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:54:28,904 - day_to_1min_continuous - INFO - exclude holiday: 2012-05-07 23:59:00, symbol: CTV12
2020-06-28 21:54:28,929 - day_to_1min_continuous - INFO - weekday:1
2020-06-28 21:54:28,931 - day_to_1min_continuous - INFO - exclude holiday: 2012-05-08 23:59:00, symbol: CTV12
2020-06-28 21:54:28,959 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 21:54:28,961 - day_to_1min_continuous - INFO - exclude holiday: 2012-05-09 23:59:00, symbol: CTV12
2020-06-28 21:54:28,985 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 21:54:28,987 - day_to_1min_continuous - INFO - exclude holiday: 2012-05-10 23:59:00, symbol: CTV12
2020-06-28 21:54:29,013 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 21:54:29,014 - day_to_1min_cont

2020-06-28 21:56:27,292 - day_to_1min_continuous - INFO - process: 2000/2172
2020-06-28 21:58:05,172 - day_to_1min_continuous - INFO - total time used is: 177.70735049247742
2020-06-28 21:58:05,178 - day_to_1min_continuous - INFO - Starting processing data for SB
2020-06-28 21:58:05,302 - day_to_1min_continuous - INFO - Reading all data to cache.
2020-06-28 21:58:23,673 - day_to_1min_continuous - INFO - process: 0/2540
2020-06-28 21:58:47,985 - day_to_1min_continuous - INFO - process: 500/2540
2020-06-28 21:59:14,219 - day_to_1min_continuous - INFO - process: 1000/2540
2020-06-28 21:59:39,287 - day_to_1min_continuous - INFO - process: 1500/2540
2020-06-28 22:00:05,045 - day_to_1min_continuous - INFO - process: 2000/2540
2020-06-28 22:00:27,925 - day_to_1min_continuous - INFO - process: 2500/2540
2020-06-28 22:02:16,704 - day_to_1min_continuous - INFO - total time used is: 233.2636559009552
2020-06-28 22:02:16,707 - day_to_1min_continuous - INFO - Starting processing data for EUR
2020-0

2020-06-28 22:15:38,768 - day_to_1min_continuous - INFO - process: 1500/3970
2020-06-28 22:15:58,842 - day_to_1min_continuous - INFO - process: 2000/3970
2020-06-28 22:16:09,776 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 22:16:09,778 - day_to_1min_continuous - INFO - exclude holiday: 2014-12-25 17:00:00, symbol: GBPH15
2020-06-28 22:16:10,022 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 22:16:10,025 - day_to_1min_continuous - INFO - exclude holiday: 2015-01-01 17:00:00, symbol: GBPH15
2020-06-28 22:16:20,325 - day_to_1min_continuous - INFO - process: 2500/3970
2020-06-28 22:16:37,656 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 22:16:37,658 - day_to_1min_continuous - INFO - exclude holiday: 2016-12-26 17:00:00, symbol: GBPH17
2020-06-28 22:16:37,888 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 22:16:37,891 - day_to_1min_continuous - INFO - exclude holiday: 2017-01-02 17:00:00, symbol: GBPH17
2020-06-28 22:16:42,591 - day_to_1min_continuous - IN

2020-06-28 22:26:10,659 - day_to_1min_continuous - INFO - exclude holiday: 2013-04-29 17:00:00, symbol: CADM14
2020-06-28 22:26:12,131 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 22:26:12,133 - day_to_1min_continuous - INFO - exclude holiday: 2013-07-08 17:00:00, symbol: CADM14
2020-06-28 22:26:16,436 - day_to_1min_continuous - INFO - weekday:1
2020-06-28 22:26:16,438 - day_to_1min_continuous - INFO - exclude holiday: 2014-03-11 17:00:00, symbol: CADH15
2020-06-28 22:26:19,043 - day_to_1min_continuous - INFO - process: 1000/1994
2020-06-28 22:26:20,304 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 22:26:20,305 - day_to_1min_continuous - INFO - exclude holiday: 2014-12-04 17:00:00, symbol: CADU15
2020-06-28 22:26:25,383 - day_to_1min_continuous - INFO - weekday:3
2020-06-28 22:26:25,385 - day_to_1min_continuous - INFO - exclude holiday: 2015-11-05 17:00:00, symbol: CADZ16
2020-06-28 22:26:25,510 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 22:26:25,512 - da

2020-06-28 22:31:37,051 - day_to_1min_continuous - INFO - weekday:1
2020-06-28 22:31:37,053 - day_to_1min_continuous - INFO - exclude holiday: 2019-01-01 17:00:00, symbol: GCG19
2020-06-28 22:31:40,506 - day_to_1min_continuous - INFO - process: 3500/3874
2020-06-28 22:31:52,245 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 22:31:52,246 - day_to_1min_continuous - INFO - exclude holiday: 2019-12-25 17:00:00, symbol: GCG20
2020-06-28 22:31:52,594 - day_to_1min_continuous - INFO - weekday:2
2020-06-28 22:31:52,596 - day_to_1min_continuous - INFO - exclude holiday: 2020-01-01 17:00:00, symbol: GCG20
2020-06-28 22:31:59,807 - day_to_1min_continuous - INFO - weekday:6
2020-06-28 22:31:59,809 - day_to_1min_continuous - INFO - exclude holiday: 2020-06-14 23:59:00, symbol: GCQ20
2020-06-28 22:31:59,825 - day_to_1min_continuous - INFO - weekday:0
2020-06-28 22:31:59,828 - day_to_1min_continuous - INFO - exclude holiday: 2020-06-15 17:00:00, symbol: GCQ20
2020-06-28 22:31:59,845 - day_to_

In [158]:
for handler in logger.handlers[:]:
    handler.close()
    logger.removeHandler(handler)